In [0]:
!pip install shap
!pip install importlib
!pip install lifelines
!pip install plotly

In [0]:
from google.colab import auth
from google.colab import drive
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

pd.set_option('display.max_colwidth', -1)

auth.authenticate_user()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!git clone https://github.com/gowun/MedicalDataScience.git

Cloning into 'MedicalDataScience'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 361 (delta 43), reused 80 (delta 29), pack-reused 265
Receiving objects: 100% (361/361), 13.98 MiB | 28.01 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [0]:
from MedicalDataScience.modeling_tools import  utils as ut
from MedicalDataScience.modeling_tools import clustering as cl
from MedicalDataScience.modeling_tools import preprocessing as pp
from MedicalDataScience.modeling_tools import tree_modeling as tm
from MedicalDataScience.modeling_tools import linear_modeling as lm
from MedicalDataScience.modeling_tools import survival_analysis as sa
from MedicalDataScience.modeling_tools import metrics as mt
from MedicalDataScience.modeling_tools import figure_eda as fe

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
from MedicalDataScience.BladderCancer_Shin.BladderCancer_Shin import BladderCancerQuantSeq

a = BladderCancerQuantSeq()
nor_df = a.nor_df

The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
Trying to unpickle estimator DecisionTreeClassifier from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator RandomForestClassifier from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[

In [0]:
home_path = '/content/gdrive/My Drive/'
nor_out_df = ut.load_data(home_path + 'Prof.Shin/nor_out_df.pkl', 'pickle')

In [0]:
nor_df.head()

,DDX11L1,WASH7P,FAM138A,OR4F5,LOC729737,LOC100133331,LOC100288069,FAM87B,LINC00115,LINC01128,FAM41C,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,C1orf159,LINC01342,TTLL10,TNFRSF18,TNFRSF4,SDF4,B3GALT6,FAM132A,UBE2J2,SCNN1D,ACAP3,PUSL1,CPSF3L,CPTP,TAS1R3,DVL1,MXRA8,AURKAIP1,CCNL2,LOC148413,...,LINC00278,PCDH11Y,TTTY2,TTTY2B,TTTY8,TTTY8B,TBL1Y,PRKY,GYG2P1,TTTY15,USP9Y,DDX3Y,UTY,TMSB4Y,NLGN4Y,NLGN4Y-AS1,FAM224B,FAM224A,HSFY2,HSFY1,TTTY14,CD24,BCORP1,TXLNGY,KDM5D,EIF1AY,RPS4Y2,PRORY,RBMY1B,RBMY1A1,RBMY1D,RBMY1E,LOC101929148,RBMY1F,RBMY1J,RBMY2FP,DAZ4,DAZ1,DAZ3,DAZ2
AMC_17S_20395_R,0.096870,-0.027812,-0.576902,-0.258199,0.695031,1.386653,-0.667293,-0.377939,-0.766898,0.427559,-0.575622,-0.666675,-0.187851,-0.196506,-1.173010,-1.475282,0.829431,-0.276242,1.186650,-0.192512,-0.217301,-0.576160,-0.574756,-0.577122,-0.750190,0.0,0.545182,-0.475447,0.0,-0.326226,0.314366,1.888889,1.742026,1.027350,-0.258199,0.0,-2.351001,-0.933429,-0.210521,0.032882,...,-0.258199,-0.741577,-0.258199,-0.258199,-0.258199,-0.377964,-0.48028,-1.265569,-0.855203,-1.291503,-1.298007,-1.007110,1.143612,-0.812176,-0.944450,-0.476396,-0.377963,-0.477106,-0.471185,-0.377964,-0.727693,0.558188,-0.577234,-1.038252,-1.400986,-1.097870,-0.576311,-0.258199,-0.258199,-0.258199,-0.258199,-0.258199,-0.480265,-0.258199,-0.258199,-0.673564,-0.479693,-0.478318,-0.478659,-0.478612
AMC_17S_28715_R,0.267636,0.325767,-0.576902,-0.258199,2.305207,0.504856,-0.667293,-0.377939,1.488110,-0.007214,-0.575622,-0.666675,-0.331596,-0.339138,1.274672,0.122451,1.363468,0.889703,-0.697791,1.768714,0.622550,-0.576160,1.847744,1.811882,1.422297,0.0,1.235307,2.302091,0.0,-1.169943,1.103174,0.836984,0.878318,1.222769,-0.258199,0.0,-0.210416,1.448491,1.409392,1.079837,...,-0.258199,-0.186918,-0.258199,-0.258199,-0.258199,-0.377964,-0.48028,-1.265569,-0.855203,-0.362152,-1.298007,-1.007110,-1.307403,-1.554274,-0.944450,-0.476396,-0.377963,-0.477106,-0.471185,-0.377964,-0.727693,-0.499764,-0.577234,-1.038252,-1.156520,-0.692314,-0.576311,-0.258199,-0.258199,-0.258199,-0.258199,-0.258199,-0.480265,-0.258199,-0.258199,-0.673564,-0.479693,-0.478318,-0.478659,-0.478612
AMC_13S_50750_R,-1.724713,-1.777444,-0.576902,-0.258199,-2.088541,-2.099319,-0.667293,-0.377939,-0.766898,-1.582314,-0.575622,-0.666675,-2.176653,-2.169841,-1.173010,-1.475282,-1.126776,-1.582418,0.096529,-0.507645,-1.652807,-0.576160,-0.574756,-0.577122,-0.750190,0.0,-1.832753,-0.475447,0.0,-0.326226,-1.705273,-2.094466,-2.023759,-1.680126,-0.258199,0.0,1.550942,-2.361415,-1.275003,-2.184520,...,-0.258199,-1.215416,-0.258199,-0.258199,-0.258199,-0.377964,-0.48028,-1.464752,1.717337,-1.291503,-1.298007,-1.007110,-1.307403,0.507363,-0.944450,-0.476396,-0.377963,-0.477106,-0.471185,-0.377964,-0.727693,-2.458275,-0.577234,-1.038252,-1.400986,-1.097870,-0.576311,-0.258199,-0.258199,-0.258199,-0.258199,-0.258199,-0.480265,-0.258199,-0.258199,-0.673564,-0.479693,-0.478318,-0.478659,-0.478612
AMC_14S_58173_R,0.363426,0.218160,-0.576902,-0.258199,-1.571208,-1.169091,-0.667293,-0.377939,-0.766898,-1.480756,-0.575622,1.663064,0.870635,1.073989,-1.173010,0.950033,-1.126776,0.517449,0.409785,1.383711,-1.756605,-0.576160,-0.574756,-0.577122,-0.750190,0.0,-1.922235,-0.475447,0.0,1.800552,-1.603747,0.171476,0.259179,-1.680126,-0.258199,0.0,-0.550580,-0.166609,-0.938309,0.867940,...,-0.258199,-0.741577,-0.258199,-0.258199,-0.258199,-0.377964,-0.48028,0.986395,0.866280,-1.291503,1.492849,-1.007110,0.937183,-0.812176,1.718170,-0.476396,-0.377963,-0.477106,-0.471185,-0.377964,1.469194,0.077128,-0.577234,-1.038252,0.696352,1.050934,1.811978,-0.258199,-0.258199,-0.258199,-0.258199,-0.258199,2.044678,-0.258199,-0.258199,1.443015,2.199448,2.265713,2.287662,2.290853
AMC_16S_63679_R,0.044519,0.066995,-0.576902,-0.258199,-0.065888,0.363932,1.730012,2.612550,1.343008,-0.277298,-0.575622,-0.666675,-0.395486,-0.397943,0.848818,0.675410,0.642644,0.094939,1.005401,0.177478,0.380551,1.736856,1.402248,1.734612,1.362471,0.0,-0.471111,-0.475447,0.0,-1.

In [0]:
nor_out_df.head()

,7A5,A1BG,A1CF,A26C3,A2BP1,A2LD1,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAA1,AAAS,AACS,AADAC,AADACL1,AADACL2,AADACL4,AADAT,AAGAB,AAK1,AAMP,AANAT,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA11,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,...,ZPBP2,ZPLD1,ZRANB1,ZRANB2,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN12,ZSCAN12L1,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN22,ZSCAN23,ZSCAN29,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM7,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,dJ222E13.2,dJ341D10.1,psiTPTE22
NR,0.649196,0.201291,-1.174141,-0.619279,0.805849,-0.084496,-0.664402,0.555338,-1.183071,0.262235,-0.952188,-1.263826,0.020824,-0.515530,-0.826138,-0.018134,0.304830,-0.238414,0.954054,-0.093202,-0.571678,-0.902207,-0.556867,-0.949911,-0.476757,0.829577,-0.804497,0.677622,-0.161664,1.491310,-0.192169,-0.685605,-0.734758,-0.454038,0.271084,0.913557,-0.129262,-0.416635,-1.705930,0.361203,...,-0.953824,-1.045454,0.353032,-0.116228,-0.478195,0.396110,0.374781,-0.518348,0.833998,0.409270,0.753789,-0.261749,1.230259,0.560313,0.032502,1.285346,-0.831857,-1.187900,-1.643886,-0.125623,-0.056539,0.581091,-0.180906,0.429108,-2.284842,1.115083,-0.324992,0.882412,0.369990,0.339467,0.721070,-1.166586,-0.489745,-1.133901,0.188284,-0.397043,-0.899218,0.407656,-0.146938,-2.296636
NR,0.114779,-1.159285,-0.180102,0.867881,0.537841,0.557135,1.832099,-1.458327,-1.062275,0.575726,-0.252706,-0.069350,0.945360,0.394340,-1.022307,0.313332,-2.001228,0.123077,0.190899,0.233298,0.774162,0.667948,0.036824,0.681156,0.532468,-0.507982,-0.005784,-0.335416,-0.041473,-0.811190,-1.687246,1.452122,1.166559,0.455618,0.589272,-0.327029,1.348552,0.720995,1.126490,-0.162166,...,0.648188,-0.080352,-0.370109,-0.323844,0.541405,-0.386370,-0.799085,0.226204,-0.852224,-1.692196,-0.379959,-0.024254,-0.197811,0.472206,-1.870788,-0.580498,-0.069906,0.112626,0.847367,-0.289699,0.919617,-0.349897,-0.988891,-1.250326,0.570061,-1.778510,-0.229984,-0.899817,-1.481839,0.687805,0.752366,0.116279,-0.985603,1.125292,1.157553,1.156576,-0.449609,-0.098299,0.476767,1.748430
NR,-1.181776,2.005020,1.709269,0.673867,1.623129,-1.702738,-1.709088,-0.728173,1.817806,-2.079186,1.831065,1.714553,-1.877251,-2.603146,-0.407727,-0.873554,1.636331,2.232494,0.459331,1.485781,-2.014773,-1.764356,0.948734,-2.146726,-2.703274,2.407801,-1.507560,1.709631,0.453529,-0.886806,0.917405,-0.574544,-1.981577,1.038691,2.172138,-0.102603,-1.008648,-1.966813,-1.247402,0.491281,...,0.336601,-0.394073,-0.802503,2.207229,-2.090714,2.299980,2.694642,-1.223056,-0.400728,-1.058943,-1.964761,1.219563,-2.061438,-0.272006,-2.015207,-1.011239,0.069585,-0.603209,-0.262547,-0.943088,-2.280808,0.706156,1.686321,1.728368,0.671369,1.103435,-0.350069,-1.129995,0.376345,-1.399723,-1.995866,0.955610,-0.846160,0.153100,-1.371636,-2.662549,2.098176,1.990049,1.114509,-0.319927
NR,-0.069514,0.155835,0.956740,0.299053,0.022291,-0.558536,-1.745926,-0.031545,1.039554,-1.093699,0.884160,0.644314,-0.878778,-0.209711,0.136835,-0.539264,0.194316,-2.454365,0.114856,-0.643796,-1.530186,-1.491629,1.170413,-0.358769,-0.221160,1.153746,2.410912,-1.767296,0.778619,1.758240,1.308287,-1.756105,-0.871042,0.757001,0.504222,0.534190,0.061462,0.827361,-0.556211,0.554343,...,2.024242,0.241874,1.983262,1.579891,0.495313,1.627564,0.312469,-1.419115,-2.194869,-0.159610,-0.023615,1.650652,-0.474096,-1.550840,1.051213,-1.600271,1.074668,-1.192600,-0.150402,0.137766,-1.021933,-0.111330,-0.399722,1.667985,0.763829,0.445925,-0.196770,-1.272187,-0.530175,-1.694277,-2.018312,0.477458,1.341588,-1.582307,-1.261981,1.099561,1.049088,-1.254892,-0.431031,0.497969
NR,0.412464,0.531886,-0.164891,1.584439,0.175991,0.097389,-1.835103,0.995682,-1.221547,-1.216246,1.123431,1.497871,-0.618274,-0.122281,-0.237998,0.110267,0.227850,1.463740,0.189973,1.094407,1.208485,-1.166041,-0.380461,-1.491128,-1.570966,0.387301,-1.097804,0.278757,1.831442,0.792339,1.062917,-1.704342,-0.706755,-0.341710,0.725596,1.390280,-0.450482,1.251419,-0.416314,-1.347321,...,0.4612

In [0]:
y_in = list(map(lambda x: 0 if x.endswith('_NR') else 1, nor_df.index))
y_out = list(map(lambda x: 0 if x == 'NR' else 1, nor_out_df.index))

In [0]:
y_in, y_out

([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [0]:
org_lb = ['AMC_R'] * 8 + ['AMC_NR'] * 8
out_lb = list(map(lambda x: 'Outside_' + x, nor_out_df.index))

In [0]:
org_classifiers = ut.load_data(home_path + 'Prof.Shin/added_classifiers.csv', 'csv')

In [0]:
org_classifiers = org_classifiers.drop(['gowun_1', 'gowun_2'], 1)

In [0]:
 class_dict = dict()
 for c in org_classifiers.columns:
   class_dict[c] = list(pd.value_counts(org_classifiers[c]).keys())

In [0]:
another = ut.load_data(home_path + 'Prof.Shin/Bladder cancer classifier_Basal, Luminal, p53.csv', 'csv')

In [0]:
class_dict[another.columns[-1]] = list(another[another.columns[-1]])

In [0]:
class_dict.keys()

dict_keys(['Metacore&GSEA target genes', 'GSEA_R enrichment_all', 'GSEA_R enrichment_intersection', 'BC Basal', 'BC basal-9gene', 'BC Luminal', 'BC Luminal-16gene', 'BC P53 pathway', 'BC P53like', 'BC P63 pathway', 'Basal-squamous_ref.eau-cancer genome', 'Luminal_ref.eau-cancer genome', 'Luminal-infiltrated_ref.eau-cancer genome', 'Luminal-papillary_ref.eau-cancer genome', 'Neuronal_ref.eau-cancer genome', 'p53-ness in the chemoresistant tumors'])

In [0]:
major_interests = ['BC Basal', 'BC Luminal', 'BC P53like', 'BC P53 pathway', 'p53-ness in the chemoresistant tumors']

In [0]:
def confirm_by_ttest(arr1, arr2, pvalue=0.05):
  tmp = ttest_ind(arr1, arr2, equal_var=False)
  if tmp.pvalue <= pvalue:
    differ = True
  else:
    differ = False
  return differ, tmp

In [0]:
def modeling_with_various_features(X_tr, y_tr, X_ts, y_ts, fts_dict, md_mode):
  result = dict()
  result['remained_features'] = dict()
  result['models'] = dict()
  result['scores_tr_val'] = dict()
  result['ttest_vals'] = dict()
  result['best_classifiers'] = []

  perf_tr = []
  perf_ts = []
  for k in fts_dict.keys():
    filtered = list(filter(lambda x: x in X_tr.columns and x in X_ts.columns, fts_dict[k]))
    filtered = list(filter(lambda x: confirm_by_ttest(X_tr[x].values, X_ts[x].values)[0] == False, filtered))

    if len(filtered) > 10:
      filtered = tm.select_important_variables([X_tr[filtered], y_tr], 30, 10, 3, 3, 0.8)
    result['remained_features'][k] = filtered
    print(len(fts_dict[k]), len(result['remained_features'][k]))
    X1 = X_tr[filtered]
    X2 = X_ts[filtered]

    if md_mode == 'logistic':
      tmp = lm.logiReg_model_with_performance([X1, y_tr], 10, class_weight='balanced')
    elif md_mode == 'decision':
      tmp = tm.tree_model_with_performance([X1, y_tr], 3, 3, class_weight='balanced')
    elif md_mode == 'random':
      tmp = tm.random_forest_with_performance([X1, y_tr], 50, 3, 3)
    
    prob_tr = tmp['model'].predict_proba(X1)[:, 1]
    pred_ts = tmp['model'].predict(X2)
    prob_ts = tmp['model'].predict_proba(X2)[:, 1]

    result['scores_tr_val'][k] = [prob_tr, prob_ts]

    ### 스코어 검증
    div_tr = []
    div_ts = []
    for i in range(2):
      div_tr.append(prob_tr[np.array(y_tr) == i])
      div_ts.append(prob_ts[np.array(y_ts) == i])
    # 1. 동일데이터 내 R vs. NR 차이가 유효한가
    # 2. R 끼리 유사한가
    # 3. NR 끼리 유사한가
    result['ttest_vals'][k] = [confirm_by_ttest(div_tr[0], div_tr[1])[0], confirm_by_ttest(div_ts[0], div_ts[1])[0], not confirm_by_ttest(div_tr[0], div_ts[0])[0], not confirm_by_ttest(div_tr[1], div_ts[1])[0]]
    if sum(result['ttest_vals'][k]) == 4:
      result['best_classifiers'].append(k)

    result['models'][k] = tmp
    perf_tr.append(tmp['performance'])
    perf_ts.append(tm.compute_performance(y_ts, pred_ts, prob_ts))
  compare_tr = pd.DataFrame(perf_tr, index=list(fts_dict.keys()))
  compare_ts = pd.DataFrame(perf_ts, index=list(fts_dict.keys()))

  if len(result['best_classifiers']) == 0:
    sum_arr = list(map(lambda x: sum(x), result['ttest_vals'].values()))
    max_v = max(sum_arr)
    for k in result['ttest_vals'].keys():
      if sum(result['ttest_vals'][k]) == max_v:
        result['best_classifiers'].append(k)
  return result, pd.concat([compare_tr, compare_ts], 1)

In [0]:
### Logistic Regression Modeling on AMC data
lrs, com_lr = modeling_with_various_features(nor_df, y_in, nor_out_df, y_out, class_dict, 'logistic')

{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.88}
31 5
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.924179165723822}
{'AUC': 0.5784313725490196, 'accuracy': 0.5217391304347826, 'precision': 0.3333333333333333, 'recall': 0.8333333333333334, 'f1': 0.47619047619047616, 'mcc': 0.22596922689443352, 'R2': -0.7869364183533163}
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.7804698631822777}
38 10
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.8779344359993956}
{'AUC': 0.5490196078431372, 'accuracy': 0.391304347826087, 'precision': 0.21428571428571427, 'recall': 0.5, 'f1': 0.3, 'mcc': -0.13231403100624078, 'R2': -0.7484566110476458}
{'AUC': 0.984375, 'accuracy': 0.875, 'precision': 0.875, 'recall': 0.875, 'f1': 0.875, 'mcc': 0.75, 'R2': 0.718732907157583}
11 4
{'AUC': 1.0, 'accuracy': 0.9375, 'precision': 

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

{'AUC': 0.4607843137254902, 'accuracy': 0.5217391304347826, 'precision': 0.3076923076923077, 'recall': 0.6666666666666666, 'f1': 0.42105263157894735, 'mcc': 0.1215783580010781, 'R2': -0.7548700948243774}
9 9
{'AUC': 0.90625, 'accuracy': 0.75, 'precision': 0.75, 'recall': 0.75, 'f1': 0.75, 'mcc': 0.5, 'R2': 0.4073083640601953}
{'AUC': 0.5294117647058824, 'accuracy': 0.4782608695652174, 'precision': 0.2857142857142857, 'recall': 0.6666666666666666, 'f1': 0.4, 'mcc': 0.07056748320332842, 'R2': -0.5687608650309426}
{'AUC': 0.96875, 'accuracy': 0.9375, 'precision': 1.0, 'recall': 0.875, 'f1': 0.9333333333333333, 'mcc': 0.8819171036881969, 'R2': 0.5401004378223955}
19 10
{'AUC': 0.9375, 'accuracy': 0.9375, 'precision': 0.8888888888888888, 'recall': 1.0, 'f1': 0.9411764705882353, 'mcc': 0.8819171036881969, 'R2': 0.573702090931214}
{'AUC': 0.49019607843137253, 'accuracy': 0.43478260869565216, 'precision': 0.18181818181818182, 'recall': 0.3333333333333333, 'f1': 0.23529411764705885, 'mcc': -0.1

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

{'AUC': 1.0, 'accuracy': 0.9375, 'precision': 1.0, 'recall': 0.875, 'f1': 0.9333333333333333, 'mcc': 0.8819171036881969, 'R2': 0.7221611810658775}
96 12
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.7643158734216489}
{'AUC': 0.3333333333333333, 'accuracy': 0.391304347826087, 'precision': 0.21428571428571427, 'recall': 0.5, 'f1': 0.3, 'mcc': -0.13231403100624078, 'R2': -1.1594879660668105}
{'AUC': 0.921875, 'accuracy': 0.875, 'precision': 0.875, 'recall': 0.875, 'f1': 0.875, 'mcc': 0.75, 'R2': 0.46430044171468077}
19 10
{'AUC': 1.0, 'accuracy': 0.9375, 'precision': 1.0, 'recall': 0.875, 'f1': 0.9333333333333333, 'mcc': 0.8819171036881969, 'R2': 0.7293745765977122}
{'AUC': 0.5098039215686274, 'accuracy': 0.5217391304347826, 'precision': 0.3076923076923077, 'recall': 0.6666666666666666, 'f1': 0.42105263157894735, 'mcc': 0.1215783580010781, 'R2': -0.8133298388621488}
{'AUC': 0.984375, 'accuracy': 0.9375, 'precision': 1.0, 'recall': 0.875, 'f1

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

96 12
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.9014307269463802}
{'AUC': 0.6862745098039216, 'accuracy': 0.5652173913043478, 'precision': 0.3333333333333333, 'recall': 0.6666666666666666, 'f1': 0.4444444444444444, 'mcc': 0.17236256333167305, 'R2': -0.4721005855112461}
{'AUC': 0.9375, 'accuracy': 0.8125, 'precision': 0.7777777777777778, 'recall': 0.875, 'f1': 0.823529411764706, 'mcc': 0.629940788348712, 'R2': 0.5265283542942032}
44 12
{'AUC': 0.84375, 'accuracy': 0.8125, 'precision': 0.7777777777777778, 'recall': 0.875, 'f1': 0.823529411764706, 'mcc': 0.629940788348712, 'R2': 0.33889762859080264}
{'AUC': 0.42156862745098034, 'accuracy': 0.391304347826087, 'precision': 0.21428571428571427, 'recall': 0.5, 'f1': 0.3, 'mcc': -0.13231403100624078, 'R2': -0.5641197387620265}
{'AUC': 0.953125, 'accuracy': 0.875, 'precision': 0.8, 'recall': 1.0, 'f1': 0.888888888888889, 'mcc': 0.7745966692414834, 'R2': 0.5234730717373948}
71 14
{'AUC': 1.0, '

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

23 10
{'AUC': 0.984375, 'accuracy': 0.875, 'precision': 0.875, 'recall': 0.875, 'f1': 0.875, 'mcc': 0.75, 'R2': 0.6198401747352572}
{'AUC': 0.6666666666666667, 'accuracy': 0.6086956521739131, 'precision': 0.36363636363636365, 'recall': 0.6666666666666666, 'f1': 0.4705882352941177, 'mcc': 0.22407133233117496, 'R2': -0.3793375819931917}
{'AUC': 1.0, 'accuracy': 0.9375, 'precision': 0.8888888888888888, 'recall': 1.0, 'f1': 0.9411764705882353, 'mcc': 0.8819171036881969, 'R2': 0.5860901942275586}
97 15
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.8364832791786708}
{'AUC': 0.2647058823529412, 'accuracy': 0.391304347826087, 'precision': 0.1, 'recall': 0.16666666666666666, 'f1': 0.125, 'mcc': -0.3213142318599921, 'R2': -1.1639805474407074}
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.8034459541724499}
173 10
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2'

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [0]:
lrs['best_classifiers'], lrs['ttest_vals']

(['BC basal-9gene', 'Luminal_ref.eau-cancer genome'],
 {'BC Basal': [True, False, False, True],
  'BC Luminal': [True, False, False, True],
  'BC Luminal-16gene': [True, False, False, True],
  'BC P53 pathway': [True, False, False, False],
  'BC P53like': [True, False, False, True],
  'BC P63 pathway': [True, False, False, True],
  'BC basal-9gene': [True, False, True, True],
  'Basal-squamous_ref.eau-cancer genome': [True, False, False, True],
  'GSEA_R enrichment_all': [True, False, False, True],
  'GSEA_R enrichment_intersection': [True, False, False, True],
  'Luminal-infiltrated_ref.eau-cancer genome': [True, False, False, True],
  'Luminal-papillary_ref.eau-cancer genome': [True, False, False, True],
  'Luminal_ref.eau-cancer genome': [True, False, True, True],
  'Metacore&GSEA target genes': [True, False, False, True],
  'Neuronal_ref.eau-cancer genome': [True, False, False, False],
  'p53-ness in the chemoresistant tumors': [True, False, False, True]})

In [0]:
### Decision Tree Modeling on AMC data
dts, com_dt = modeling_with_various_features(nor_df, y_in, nor_out_df, y_out, class_dict, 'decision')

{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.88}
31 5
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 1.0}
{'AUC': 0.4313725490196078, 'accuracy': 0.4782608695652174, 'precision': 0.2, 'recall': 0.3333333333333333, 'f1': 0.25, 'mcc': -0.1215783580010781, 'R2': -1.7058823529411762}
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.7804698631822777}
38 10
{'AUC': 0.953125, 'accuracy': 0.875, 'precision': 0.875, 'recall': 0.875, 'f1': 0.875, 'mcc': 0.75, 'R2': 0.6666666666666666}
{'AUC': 0.6029411764705883, 'accuracy': 0.6521739130434783, 'precision': 0.4, 'recall': 0.6666666666666666, 'f1': 0.5, 'mcc': 0.27789338971674993, 'R2': -0.5784313725490193}
{'AUC': 0.984375, 'accuracy': 0.875, 'precision': 0.875, 'recall': 0.875, 'f1': 0.875, 'mcc': 0.75, 'R2': 0.718732907157583}
11 4
{'AUC': 0.953125, 'accuracy': 0.875, 'precision': 0.875, 'recall': 0.875, 'f1

In [0]:
dts['best_classifiers'], dts['ttest_vals']

(['GSEA_R enrichment_all',
  'GSEA_R enrichment_intersection',
  'BC Basal',
  'BC basal-9gene',
  'BC Luminal',
  'BC Luminal-16gene',
  'BC P53 pathway',
  'BC P53like',
  'BC P63 pathway',
  'Basal-squamous_ref.eau-cancer genome',
  'Luminal_ref.eau-cancer genome',
  'Luminal-infiltrated_ref.eau-cancer genome',
  'Luminal-papillary_ref.eau-cancer genome',
  'Neuronal_ref.eau-cancer genome',
  'p53-ness in the chemoresistant tumors'],
 {'BC Basal': [True, False, False, True],
  'BC Luminal': [True, False, False, True],
  'BC Luminal-16gene': [True, False, True, False],
  'BC P53 pathway': [True, False, False, True],
  'BC P53like': [True, False, False, True],
  'BC P63 pathway': [True, False, True, False],
  'BC basal-9gene': [True, False, False, True],
  'Basal-squamous_ref.eau-cancer genome': [True, False, False, True],
  'GSEA_R enrichment_all': [True, False, False, True],
  'GSEA_R enrichment_intersection': [True, False, False, True],
  'Luminal-infiltrated_ref.eau-cancer genome'

In [0]:
### Random Forest Modeling on AMC data
rfs, com_rf = modeling_with_various_features(nor_df, y_in, nor_out_df, y_out, class_dict, 'random')

{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.88}
31 5
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.9475955867850099}
{'AUC': 0.5147058823529412, 'accuracy': 0.43478260869565216, 'precision': 0.18181818181818182, 'recall': 0.3333333333333333, 'f1': 0.23529411764705885, 'mcc': -0.17236256333167305, 'R2': -0.9472576584677261}
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.7804698631822777}
38 10
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.7833812147654661}
{'AUC': 0.607843137254902, 'accuracy': 0.6086956521739131, 'precision': 0.3333333333333333, 'recall': 0.5, 'f1': 0.4, 'mcc': 0.13231403100624078, 'R2': -0.3812567731239578}
{'AUC': 0.984375, 'accuracy': 0.875, 'precision': 0.875, 'recall': 0.875, 'f1': 0.875, 'mcc': 0.75, 'R2': 0.718732907157583}
11 4
{'AUC': 1.0, 'accuracy': 0.9375, 'precision'

In [0]:
rfs['best_classifiers'], rfs['ttest_vals']

(['GSEA_R enrichment_all',
  'GSEA_R enrichment_intersection',
  'BC basal-9gene',
  'BC P63 pathway',
  'Basal-squamous_ref.eau-cancer genome',
  'Luminal_ref.eau-cancer genome',
  'Luminal-infiltrated_ref.eau-cancer genome',
  'Luminal-papillary_ref.eau-cancer genome'],
 {'BC Basal': [True, False, False, False],
  'BC Luminal': [True, False, False, False],
  'BC Luminal-16gene': [True, False, False, False],
  'BC P53 pathway': [True, False, False, False],
  'BC P53like': [True, False, False, False],
  'BC P63 pathway': [True, False, False, True],
  'BC basal-9gene': [True, False, False, True],
  'Basal-squamous_ref.eau-cancer genome': [True, False, False, True],
  'GSEA_R enrichment_all': [True, False, False, True],
  'GSEA_R enrichment_intersection': [True, False, False, True],
  'Luminal-infiltrated_ref.eau-cancer genome': [True, False, False, True],
  'Luminal-papillary_ref.eau-cancer genome': [True, False, False, True],
  'Luminal_ref.eau-cancer genome': [True, False, False, True

In [0]:
lrs1, com_lr1 = modeling_with_various_features(nor_out_df, y_out, nor_df, y_in, class_dict, 'logistic')
dts1, com_dt1 = modeling_with_various_features(nor_out_df, y_out, nor_df, y_in, class_dict, 'decision')
rfs1, com_rf1 = modeling_with_various_features(nor_out_df, y_out, nor_df, y_in, class_dict, 'random')

{'AUC': 1.0, 'accuracy': 0.9565217391304348, 'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'mcc': 0.8871510790372951, 'R2': 0.6024174580152147}
31 13
{'AUC': 0.9803921568627451, 'accuracy': 0.8695652173913043, 'precision': 0.6666666666666666, 'recall': 1.0, 'f1': 0.8, 'mcc': 0.7409585736349484, 'R2': 0.6039824016839677}
{'AUC': 0.953125, 'accuracy': 0.6875, 'precision': 1.0, 'recall': 0.375, 'f1': 0.5454545454545454, 'mcc': 0.48038446141526137, 'R2': 0.2832955037671441}
{'AUC': 0.9901960784313726, 'accuracy': 0.9565217391304348, 'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'mcc': 0.8871510790372951, 'R2': 0.543934985441632}
38 13
{'AUC': 0.9411764705882353, 'accuracy': 0.8695652173913043, 'precision': 0.6666666666666666, 'recall': 1.0, 'f1': 0.8, 'mcc': 0.7409585736349484, 'R2': 0.422511088609558}
{'AUC': 0.609375, 'accuracy': 0.5625, 'precision': 0.6, 'recall': 0.375, 'f1': 0.4615384615384615, 'mcc': 0.13483997249264842, 'R2': -

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

{'AUC': 0.234375, 'accuracy': 0.375, 'precision': 0.375, 'recall': 0.375, 'f1': 0.375, 'mcc': -0.25, 'R2': -0.9197351747686249}
9 9
{'AUC': 0.7549019607843137, 'accuracy': 0.6956521739130435, 'precision': 0.4444444444444444, 'recall': 0.6666666666666666, 'f1': 0.5333333333333333, 'mcc': 0.33519554521581, 'R2': -0.054835284219512115}
{'AUC': 0.546875, 'accuracy': 0.375, 'precision': 0.25, 'recall': 0.125, 'f1': 0.16666666666666666, 'mcc': -0.2886751345948129, 'R2': -0.14811805873967732}
{'AUC': 1.0, 'accuracy': 0.9565217391304348, 'precision': 0.8571428571428571, 'recall': 1.0, 'f1': 0.923076923076923, 'mcc': 0.8981774262781437, 'R2': 0.5313593087992232}
19 11
{'AUC': 0.9705882352941176, 'accuracy': 0.8695652173913043, 'precision': 0.6666666666666666, 'recall': 1.0, 'f1': 0.8, 'mcc': 0.7409585736349484, 'R2': 0.40861979837414153}
{'AUC': 0.453125, 'accuracy': 0.4375, 'precision': 0.42857142857142855, 'recall': 0.375, 'f1': 0.39999999999999997, 'mcc': -0.1259881576697424, 'R2': -0.545716

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

{'AUC': 0.9901960784313725, 'accuracy': 0.9565217391304348, 'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'mcc': 0.8871510790372951, 'R2': 0.6278753630455964}
96 12
{'AUC': 0.9705882352941176, 'accuracy': 0.8260869565217391, 'precision': 0.625, 'recall': 0.8333333333333334, 'f1': 0.7142857142857143, 'mcc': 0.6055975280770818, 'R2': 0.5819614460063796}
{'AUC': 0.515625, 'accuracy': 0.5625, 'precision': 0.6, 'recall': 0.375, 'f1': 0.4615384615384615, 'mcc': 0.13483997249264842, 'R2': -0.3331937685499511}
{'AUC': 0.9803921568627451, 'accuracy': 0.9130434782608695, 'precision': 0.75, 'recall': 1.0, 'f1': 0.8571428571428571, 'mcc': 0.8134892168199607, 'R2': 0.5450381403228982}
19 11
{'AUC': 0.8627450980392157, 'accuracy': 0.782608695652174, 'precision': 0.5555555555555556, 'recall': 0.8333333333333334, 'f1': 0.6666666666666667, 'mcc': 0.5380770594253792, 'R2': 0.19989369832679738}
{'AUC': 0.640625, 'accuracy': 0.5, 'precision': 0.5, 'recall': 0.375, 'f1': 0.42857

...

{'AUC': 1.0, 'accuracy': 0.9565217391304348, 'precision': 0.8571428571428571, 'recall': 1.0, 'f1': 0.923076923076923, 'mcc': 0.8981774262781437, 'R2': 0.7539851744074726}
{'AUC': 0.5625, 'accuracy': 0.6875, 'precision': 1.0, 'recall': 0.375, 'f1': 0.5454545454545454, 'mcc': 0.48038446141526137, 'R2': -0.1452772290737343}
{'AUC': 1.0, 'accuracy': 0.9565217391304348, 'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'mcc': 0.8871510790372951, 'R2': 0.6024174580152147}
31 13
{'AUC': 0.9901960784313726, 'accuracy': 0.9130434782608695, 'precision': 0.75, 'recall': 1.0, 'f1': 0.8571428571428571, 'mcc': 0.8134892168199607, 'R2': 0.8064162644843906}
{'AUC': 0.625, 'accuracy': 0.5625, 'precision': 0.5714285714285714, 'recall': 0.5, 'f1': 0.5333333333333333, 'mcc': 0.1259881576697424, 'R2': -0.4218192627824022}
{'AUC': 0.9901960784313726, 'accuracy': 0.9565217391304348, 'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'mcc': 0.8871510790372951, 'R

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
invalid value encountered in double_scalars


{'AUC': 0.640625, 'accuracy': 0.625, 'precision': 0.75, 'recall': 0.375, 'f1': 0.5, 'mcc': 0.2886751345948129, 'R2': -0.43625}
{'AUC': 1.0, 'accuracy': 0.9565217391304348, 'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'mcc': 0.8871510790372951, 'R2': 0.6024174580152147}
31 13
{'AUC': 1.0, 'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'mcc': 1.0, 'R2': 0.6627450496607418}
{'AUC': 0.75, 'accuracy': 0.6875, 'precision': 1.0, 'recall': 0.375, 'f1': 0.5454545454545454, 'mcc': 0.48038446141526137, 'R2': 0.06050575403158065}
{'AUC': 0.9901960784313726, 'accuracy': 0.9565217391304348, 'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'mcc': 0.8871510790372951, 'R2': 0.543934985441632}
38 13
{'AUC': 1.0, 'accuracy': 0.9565217391304348, 'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'mcc': 0.8871510790372951, 'R2': 0.5852497129691401}
{'AUC': 0.6875, 'accuracy': 0.5625, 'precision': 1.0, 'recall': 0.125

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
invalid value encountered in double_scalars


{'AUC': 0.484375, 'accuracy': 0.5, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'mcc': 0.0, 'R2': -0.3738264597870864}
{'AUC': 1.0, 'accuracy': 0.9565217391304348, 'precision': 0.8571428571428571, 'recall': 1.0, 'f1': 0.923076923076923, 'mcc': 0.8981774262781437, 'R2': 0.7123844047519834}
173 7
{'AUC': 1.0, 'accuracy': 0.9565217391304348, 'precision': 0.8571428571428571, 'recall': 1.0, 'f1': 0.923076923076923, 'mcc': 0.8981774262781437, 'R2': 0.8211538221027515}
{'AUC': 0.578125, 'accuracy': 0.6875, 'precision': 1.0, 'recall': 0.375, 'f1': 0.5454545454545454, 'mcc': 0.48038446141526137, 'R2': -0.19365963914783046}


In [0]:
lrs1['best_classifiers'], lrs1['ttest_vals']

(['Metacore&GSEA target genes',
  'GSEA_R enrichment_all',
  'GSEA_R enrichment_intersection',
  'BC basal-9gene',
  'BC Luminal',
  'BC P53like'],
 {'BC Basal': [True, False, False, False],
  'BC Luminal': [True, False, True, True],
  'BC Luminal-16gene': [True, False, True, False],
  'BC P53 pathway': [True, False, True, False],
  'BC P53like': [True, False, True, True],
  'BC P63 pathway': [True, False, True, False],
  'BC basal-9gene': [True, False, True, True],
  'Basal-squamous_ref.eau-cancer genome': [True, False, True, False],
  'GSEA_R enrichment_all': [True, False, True, True],
  'GSEA_R enrichment_intersection': [True, False, True, True],
  'Luminal-infiltrated_ref.eau-cancer genome': [True, False, True, False],
  'Luminal-papillary_ref.eau-cancer genome': [True, False, True, False],
  'Luminal_ref.eau-cancer genome': [True, False, True, False],
  'Metacore&GSEA target genes': [True, True, True, False],
  'Neuronal_ref.eau-cancer genome': [True, False, True, False],
  'p53-n

In [0]:
dts1['best_classifiers'], dts1['ttest_vals']

(['Metacore&GSEA target genes',
  'GSEA_R enrichment_intersection',
  'BC P63 pathway',
  'Luminal-infiltrated_ref.eau-cancer genome'],
 {'BC Basal': [True, False, True, False],
  'BC Luminal': [True, False, True, False],
  'BC Luminal-16gene': [True, False, True, False],
  'BC P53 pathway': [True, False, True, False],
  'BC P53like': [True, False, True, False],
  'BC P63 pathway': [True, False, True, True],
  'BC basal-9gene': [True, False, True, False],
  'Basal-squamous_ref.eau-cancer genome': [True, False, True, False],
  'GSEA_R enrichment_all': [True, False, True, False],
  'GSEA_R enrichment_intersection': [True, False, True, True],
  'Luminal-infiltrated_ref.eau-cancer genome': [True, False, True, True],
  'Luminal-papillary_ref.eau-cancer genome': [True, False, False, True],
  'Luminal_ref.eau-cancer genome': [True, False, True, False],
  'Metacore&GSEA target genes': [True, False, True, True],
  'Neuronal_ref.eau-cancer genome': [True, False, True, False],
  'p53-ness in the 

In [0]:
rfs1['best_classifiers'], rfs1['ttest_vals']

(['GSEA_R enrichment_intersection'],
 {'BC Basal': [True, False, False, False],
  'BC Luminal': [True, False, True, False],
  'BC Luminal-16gene': [True, False, True, False],
  'BC P53 pathway': [True, False, True, False],
  'BC P53like': [True, False, True, False],
  'BC P63 pathway': [True, False, False, False],
  'BC basal-9gene': [True, False, False, False],
  'Basal-squamous_ref.eau-cancer genome': [True, True, True, False],
  'GSEA_R enrichment_all': [True, False, True, False],
  'GSEA_R enrichment_intersection': [True, True, True, True],
  'Luminal-infiltrated_ref.eau-cancer genome': [True, False, False, False],
  'Luminal-papillary_ref.eau-cancer genome': [True, False, False, False],
  'Luminal_ref.eau-cancer genome': [True, False, True, False],
  'Metacore&GSEA target genes': [True, False, True, False],
  'Neuronal_ref.eau-cancer genome': [True, False, True, False],
  'p53-ness in the chemoresistant tumors': [True, False, True, False]})

In [0]:
def draw_box_plots(list_res, list_lb):
  for i, res in enumerate(list_res):
    for b in res['best_classifiers']:
      tmp = res['scores_tr_val'][b]
      if len(tmp[0]) == len(list_lb[0]):
        lbs = list_lb[0] + list_lb[1]
        suf = ' on AMC model'
      else:
        lbs = list_lb[1] + list_lb[0]
        suf = ' on Outside model'
      fe.plot_box(np.concatenate(tmp), 'y', lbs, b + suf)

In [0]:
draw_box_plots([lrs], [org_lb, out_lb])

In [0]:
draw_box_plots([lrs1], [org_lb, out_lb])

In [0]:
draw_box_plots([dts1], [org_lb, out_lb])

In [0]:
draw_box_plots([rfs1], [org_lb, out_lb])

In [0]:
com_lr1

,AUC,accuracy,precision,recall,f1,mcc,R2,AUC,accuracy,precision,recall,f1,mcc,R2
Metacore&GSEA target genes,0.980392,0.869565,0.666667,1.000000,0.800000,0.740959,0.603982,0.953125,0.6875,1.000000,0.375,0.545455,0.480384,0.283296
GSEA_R enrichment_all,0.941176,0.869565,0.666667,1.000000,0.800000,0.740959,0.422511,0.609375,0.5625,0.600000,0.375,0.461538,0.134840,-0.144794
GSEA_R enrichment_intersection,0.833333,0.695652,0.454545,0.833333,0.588235,0.422288,0.076922,0.703125,0.7500,0.833333,0.625,0.714286,0.516398,0.113710
BC Basal,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.621452,0.234375,0.3750,0.375000,0.375,0.375000,-0.250000,-0.919735
BC basal-9gene,0.754902,0.695652,0.444444,0.666667,0.533333,0.335196,-0.054835,0.546875,0.3750,0.250000,0.125,0.166667,-0.288675,-0.148118
BC Luminal,0.970588,0.869565,0.666667,1.000000,0.800000,0.740959,0.408620,0.453125,0.4375,0.428571,0.375,0.400000,-0.125988,-0.545716
BC Luminal-16gene,0.911765,0.826087,0.625000,0.833333,0.714286,0.605598,0.385977,0.296875,0.3750,0.333333,0.250,0.285714,-0.258199,-0.724644
BC P53 pathway,0.970588,0.826087,0.625000,0.833333,0.714286,0.605598,0.581961,0.515625,0.5625,0.600000,0.375,0.461538,0.134840,-0.333194
BC P53like,0.862745,0.782609,0.555556,0.833333,0.666667,0.538077,0.199894,0.640625,0.5000,0.500000,0.375,0.428571,0.000000,-0.235225
BC P63 pathway,0.990196,0.956522,0.857143,1.000000,0.923077,0.898177,0.645741,0.703125,0.6875,0.714286,0.625,0.666667,0.377964,0.090105


In [0]:
com_rf1

,AUC,accuracy,precision,recall,f1,mcc,R2,AUC,accuracy,precision,recall,f1,mcc,R2
Metacore&GSEA target genes,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.662745,0.750000,0.6875,1.000000,0.375,0.545455,0.480384,0.060506
GSEA_R enrichment_all,1.000000,0.956522,1.000000,0.833333,0.909091,0.887151,0.585250,0.687500,0.5625,1.000000,0.125,0.222222,0.258199,-0.022017
GSEA_R enrichment_intersection,1.000000,0.956522,1.000000,0.833333,0.909091,0.887151,0.526285,0.781250,0.7500,1.000000,0.500,0.666667,0.577350,0.167394
BC Basal,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.791697,0.312500,0.3750,0.333333,0.250,0.285714,-0.258199,-0.654701
BC basal-9gene,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.510104,0.484375,0.5000,0.500000,0.375,0.428571,0.000000,-0.082275
BC Luminal,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.593570,0.437500,0.4375,0.000000,0.000,0.000000,-0.258199,-0.366006
BC Luminal-16gene,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.667638,0.421875,0.3125,0.000000,0.000,0.000000,-0.480384,-0.412734
BC P53 pathway,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.730472,0.734375,0.5625,0.666667,0.250,0.363636,0.160128,0.033571
BC P53like,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.615047,0.546875,0.4375,0.333333,0.125,0.181818,-0.160128,-0.259229
BC P63 pathway,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.694667,0.671875,0.6250,1.000000,0.250,0.400000,0.377964,0.022654


In [0]:
rfs1['models']['GSEA_R enrichment_intersection']['feature importance']

,feature,importance
3,HIST1H2BF,0.276988
0,CD247,0.242165
4,ICAM1,0.138799
1,KIR3DL2,0.110613
2,IL1B,0.102229
6,KIR3DL1,0.051185
5,KLRD1,0.045725
7,CD3E,0.032297


In [0]:
lrs1['models']['GSEA_R enrichment_intersection']['coefficients']

,feature,coefficient
0,,-0.319841
1,CD247,0.882199
2,KIR3DL2,-0.026290
3,IL1B,0.315225
4,HIST1H2BF,0.916913
5,ICAM1,-0.410643
6,KLRD1,-0.263673
7,KIR3DL1,0.034411
8,CD3E,0.197159


In [0]:
lrs1['models']['Metacore&GSEA target genes']['coefficients']

,feature,coefficient
0,,-1.094007
1,EIF4E,-0.452328
2,ZNF615,-0.425485
3,ZNF626,-0.281397
4,ZNF592,0.199354
5,ZNF697,0.916371
6,CHKB,-0.980838
7,CCDC64,0.376643
8,RPL9,-0.441604
9,RPL26L1,-0.581620


In [0]:
pp = [np.concatenate(rfs1['scores_tr_val']['GSEA_R enrichment_intersection']), np.concatenate(lrs1['scores_tr_val']['GSEA_R enrichment_intersection']), np.concatenate(lrs1['scores_tr_val']['Metacore&GSEA target genes'])]

In [0]:
pd.DataFrame(pp, columns=out_lb + org_lb)

,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_NR,Outside_R,Outside_R,Outside_R,Outside_R,Outside_R,Outside_R,AMC_R,AMC_R,AMC_R,AMC_R,AMC_R,AMC_R,AMC_R,AMC_R,AMC_NR,AMC_NR,AMC_NR,AMC_NR,AMC_NR,AMC_NR,AMC_NR,AMC_NR
0,0.379342,0.245427,0.227777,0.249907,0.456322,0.380197,0.200855,0.026087,0.335547,0.122249,0.079109,0.188283,0.186744,0.207731,0.305335,0.461510,0.150185,0.468034,0.814242,0.573357,0.859269,0.780825,0.504884,0.055186,0.422765,0.156064,0.357670,0.663453,0.570187,0.730801,0.527447,0.305153,0.220204,0.292037,0.110636,0.261189,0.177682,0.133583,0.170853
1,0.711550,0.597668,0.113675,0.135516,0.179411,0.741147,0.317070,0.329925,0.632483,0.314260,0.126212,0.171238,0.405992,0.531327,0.134619,0.454352,0.522962,0.537819,0.714982,0.688741,0.538007,0.761496,0.493346,0.086623,0.617062,0.045762,0.698354,0.844029,0.894976,0.920220,0.401803,0.487450,0.353534,0.499456,0.163927,0.662609,0.080125,0.158602,0.231495
2,0.224968,0.028730,0.060226,0.074220,0.037867,0.231439,0.564956,0.016216,0.035056,0.232313,0.006914,0.187896,0.670294,0.071234,0.672278,0.165049,0.118325,0.634621,0.921325,0.812288,0.826564,0.786549,0.819569,0.656331,0.244706,0.955258,0.978875,0.492200,0.296618,0.122095,0.301485,0.092298,0.235776,0.039189,0.003182,0.102761,0.211755,0.049574,0.202566
